In [2]:
!pip install --upgrade 'scikit-learn' pandas numpy category_encoders 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 187.1 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [category_encoders]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.252.0 requires numpy==1.26.4, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.16.2 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.2.6 which is incompatible.


In [3]:
import os, json, time, getpass, subprocess
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from category_encoders import BinaryEncoder
from scipy.linalg import block_diag
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-pa

AttributeError: _ARRAY_API not found

In [4]:
bucket_name = "sagemaker-us-east-1-176843580427"

In [5]:
!aws s3 cp \
    --recursive \
    s3://{bucket_name}/datasets/ieee-fraud-detection/ \
    ./datasets/ieee-fraud-detection/ 

download: s3://sagemaker-us-east-1-176843580427/datasets/ieee-fraud-detection/train_identity.csv to datasets/ieee-fraud-detection/train_identity.csv
download: s3://sagemaker-us-east-1-176843580427/datasets/ieee-fraud-detection/train_transaction.csv to datasets/ieee-fraud-detection/train_transaction.csv


In [6]:
!ls

datasets		   preprocessing.ipynb	 training-job.ipynb
download-upload-ecr.ipynb  processed
lost+found		   training_config.json


In [7]:
BASE_DRIVE = "./"

RAW_DATA_PATH = os.path.join(BASE_DRIVE, 'datasets/ieee-fraud-detection/')          # put train_transaction.csv & optional train_identity.csv here
PROCESSED_DATA_PATH = os.path.join(BASE_DRIVE, 'processed/ieee-fraud-detection/')   # output used by Docker
S3_PREPROCESS_DATA_PATH = os.path.join("s3://", bucket_name, 'processed/ieee-fraud-detection/')   # output used by Docker
MODEL_OUTPUT_PATH = os.path.join(BASE_DRIVE, 'trained_models/ieee-fraud-detection/')# models written by the container

os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)
os.makedirs(MODEL_OUTPUT_PATH, exist_ok=True)

print("Paths:")
print(" RAW_DATA_PATH:", RAW_DATA_PATH)
print(" PROCESSED_DATA_PATH:", PROCESSED_DATA_PATH)
print(" MODEL_OUTPUT_PATH:", MODEL_OUTPUT_PATH)
print(" BUCKET PREPROCESS PATH", S3_PREPROCESS_DATA_PATH)

Paths:
 RAW_DATA_PATH: ./datasets/ieee-fraud-detection/
 PROCESSED_DATA_PATH: ./processed/ieee-fraud-detection/
 MODEL_OUTPUT_PATH: ./trained_models/ieee-fraud-detection/
 BUCKET PREPROCESS PATH s3://sagemaker-us-east-1-176843580427/processed/ieee-fraud-detection/


In [8]:
# ---------------------------
# Load raw data and split (time-based)
# ---------------------------
csv_path = Path(RAW_DATA_PATH) / 'train_transaction.csv'
if not csv_path.exists():
    raise FileNotFoundError(f"Place `train_transaction.csv` under {RAW_DATA_PATH} before running this cell")
train_transaction = pd.read_csv(csv_path)
identity_path = Path(RAW_DATA_PATH) / 'train_identity.csv'
train_identity = pd.read_csv(identity_path) if identity_path.exists() else None

# Optionally quick sample for debugging (set None for full)
SAMPLE_SIZE = None  # e.g., 100000 for quick run
if SAMPLE_SIZE:
    train_transaction = train_transaction.head(SAMPLE_SIZE)
    if train_identity is not None:
        train_identity = train_identity[train_identity['TransactionID'].isin(train_transaction['TransactionID'])]

In [9]:
# ---------------------------
# Merge identity and preprocess BEFORE split
# ---------------------------
print("Merging identity file...")
if train_identity is not None:
    train_merged = train_transaction.merge(train_identity, on="TransactionID", how="left")
else:
    train_merged = train_transaction.copy()

print("Replacing placeholder strings with NaN...")
train_merged.replace(["unknown", "Unknown", "UNKN"], np.nan, inplace=True)

# ---------------------------
# Identify categorical and numerical features
# ---------------------------
categorical_features = [
    c for c in train_merged.columns
    if train_merged[c].dtype == "object" or train_merged[c].nunique() < 20
]
numerical_features = [
    c for c in train_merged.columns
    if c not in categorical_features + ["isFraud"]
]

print(f"Identified features: {len(categorical_features)} categorical, {len(numerical_features)} numerical")

# ---------------------------
# Fill missing values BEFORE split
# ---------------------------
for c in categorical_features:
    train_merged[c] = train_merged[c].fillna("missing")
for c in numerical_features:
    train_merged[c] = train_merged[c].fillna(train_merged[c].median())

# ---------------------------
# Encode categoricals BEFORE split
# ---------------------------
print("Encoding categoricals on full dataset...")
from sklearn.preprocessing import LabelEncoder, StandardScaler

encoders = {}
for c in categorical_features:
    le = LabelEncoder()
    train_merged[c] = le.fit_transform(train_merged[c].astype(str))
    encoders[c] = le

# ---------------------------
# Scale numerical features BEFORE split
# ---------------------------
print("Scaling numericals on full dataset...")
scaler = StandardScaler()
train_merged[numerical_features] = scaler.fit_transform(train_merged[numerical_features].astype(float))

print("✅ Preprocessing complete on full dataset. Now performing time-based split...")

# ---------------------------
# Split (time-based AFTER preprocessing)
# ---------------------------
train_merged = train_merged.sort_values('TransactionDT').reset_index(drop=True)
n = len(train_merged)
train_end = int(n * 0.7)
test_start = train_end
train_df = train_merged.iloc[:train_end].reset_index(drop=True)
test_df = train_merged.iloc[test_start:].reset_index(drop=True)

print(f"Splits: Train={len(train_df)}, Test={len(test_df)}")
print(f"Train fraud rate={train_df['isFraud'].mean():.6f}, Test fraud rate={test_df['isFraud'].mean():.6f}")

# ---------------------------
# UTILITIES: Save functions in NGC expected structure
# ---------------------------
def save_gnn_data(output_dir, edges_df, node_features_df, node_labels_df, tx_range, split_name='train'):
    base = Path(output_dir) / 'gnn' / (split_name if split_name != 'train' else 'train_gnn')
    edge_path = base / 'edges'
    node_path = base / 'nodes'
    edge_path.mkdir(parents=True, exist_ok=True)
    node_path.mkdir(parents=True, exist_ok=True)
    edges_df.to_csv(edge_path / 'node_to_node.csv', index=False)
    node_features_df.to_csv(node_path / 'node.csv', index=False)
    node_labels_df.to_csv(node_path / 'node_label.csv', index=False)
    if split_name == 'train':
        offset_info = {'start': int(tx_range[0]), 'end': int(tx_range[1])}
        with open(node_path / 'offset_range_of_training_node.json', 'w') as f:
            json.dump(offset_info, f, indent=4)
    print(f"Saved GNN data for split={split_name} at {base}")

def save_xgb_data(output_dir, train_df, test_df, feature_cols, categorical_cols):
    xgb_path = Path(output_dir) / 'xgb'
    xgb_path.mkdir(parents=True, exist_ok=True)
    cols = list(feature_cols) + ['isFraud']
    train_df[cols].to_csv(xgb_path / 'training.csv', index=False)
    test_df[cols].to_csv(xgb_path / 'test.csv', index=False)
    cat_info = {'categorical_features': categorical_cols, 'feature_names': list(feature_cols)}
    with open(xgb_path / 'feature_info.json', 'w') as f:
        json.dump(cat_info, f, indent=4)
    print(f"Saved XGBoost data at {xgb_path}")

# ---------------------------
# Graph builder (same as before)
# ---------------------------
class GraphBuilder:
    def __init__(self, df, card_col='card1', email_col='P_emaildomain', entity_vocab=None):
        self.df = df.copy().reset_index(drop=True)
        self.df['tx_id'] = self.df.index
        self.card_col = card_col
        self.email_col = email_col
        self.entity_vocab = entity_vocab

    def build_graph(self):
        if self.entity_vocab is None:
            unique_cards = pd.Series(self.df[self.card_col].astype(str).unique())
            self.card_to_id = {card: idx for idx, card in enumerate(unique_cards)}
            unique_emails = pd.Series(self.df[self.email_col].astype(str).unique())
            self.email_to_id = {email: idx for idx, email in enumerate(unique_emails)}
            self.entity_vocab = {'card_to_id': self.card_to_id, 'email_to_id': self.email_to_id}
        else:
            self.card_to_id = self.entity_vocab['card_to_id']
            self.email_to_id = self.entity_vocab['email_to_id']
            before = len(self.df)
            self.df = self.df[
                self.df[self.card_col].astype(str).isin(self.card_to_id.keys()) &
                self.df[self.email_col].astype(str).isin(self.email_to_id.keys())
            ].reset_index(drop=True)
            self.df['tx_id'] = self.df.index
            after = len(self.df)
            print(f"Filtered unseen entities: before={before}, after={after} (dropped {before-after})")

        self.num_cards = len(self.card_to_id)
        self.num_emails = len(self.email_to_id)
        self.num_transactions = len(self.df)
        print(f"Graph nodes: {self.num_cards} cards, {self.num_emails} emails, {self.num_transactions} txs")
        return self

    def create_edges(self):
        self.df['card_id'] = self.df[self.card_col].astype(str).map(self.card_to_id)
        self.df['email_id'] = self.df[self.email_col].astype(str).map(self.email_to_id)
        self.df = self.df.dropna(subset=['card_id', 'email_id']).copy()
        self.df['card_id'] = self.df['card_id'].astype(int)
        self.df['email_id'] = self.df['email_id'].astype(int)
        card_offset = 0
        email_offset = self.num_cards
        tx_offset = self.num_cards + self.num_emails
        edges = []
        for idx, row in self.df.iterrows():
            card_node = card_offset + row['card_id']
            email_node = email_offset + row['email_id']
            tx_node = tx_offset + row['tx_id']
            edges.extend([
                [card_node, tx_node],
                [tx_node, email_node],
                [tx_node, card_node],
                [email_node, tx_node]
            ])
        return pd.DataFrame(edges, columns=['src', 'dst'])

# ---------------------------
# Feature column list
# ---------------------------
feature_cols = categorical_features + numerical_features
print(f"Feature columns count: {len(feature_cols)}")

Merging identity file...
Replacing placeholder strings with NaN...
Identified features: 171 categorical, 263 numerical
Encoding categoricals on full dataset...
Scaling numericals on full dataset...
✅ Preprocessing complete on full dataset. Now performing time-based split...
Splits: Train=413378, Test=177162
Train fraud rate=0.035169, Test fraud rate=0.034573
Feature columns count: 434


In [10]:
# ===============================
# BUILD GRAPH DATA
# ===============================
print("Building train graph...")
train_graph = GraphBuilder(train_df).build_graph()
train_edges = train_graph.create_edges()
save_gnn_data(PROCESSED_DATA_PATH, train_edges, train_df[feature_cols], train_df[['isFraud']], (0, len(train_df)), split_name='train')

print("Building test graph...")
test_graph = GraphBuilder(test_df, entity_vocab=train_graph.entity_vocab).build_graph()
test_edges = test_graph.create_edges()
save_gnn_data(PROCESSED_DATA_PATH, test_edges, test_df[feature_cols], test_df[['isFraud']], (len(train_df), len(train_df)+len(test_df)), split_name='test')

Building train graph...
Graph nodes: 12242 cards, 60 emails, 413378 txs
Saved GNN data for split=train at processed/ieee-fraud-detection/gnn/train_gnn
Building test graph...
Filtered unseen entities: before=177162, after=174985 (dropped 2177)
Graph nodes: 12242 cards, 60 emails, 174985 txs
Saved GNN data for split=test at processed/ieee-fraud-detection/gnn/test


In [11]:
# ===============================
# SAVE XGBOOST TRAIN/TEST FILES
# ===============================
save_xgb_data(PROCESSED_DATA_PATH, train_df, test_df, feature_cols, categorical_features)

print("✅ All preprocessing, graph creation, and dataset export completed successfully.")

Saved XGBoost data at processed/ieee-fraud-detection/xgb
✅ All preprocessing, graph creation, and dataset export completed successfully.


In [12]:
def print_tree(directory, prefix=""):
    """Recursively prints the directory tree starting at 'directory'."""
    # Retrieve a sorted list of entries in the directory
    entries = sorted(os.listdir(directory))
    entries_count = len(entries)
    
    for index, entry in enumerate(entries):
        path = os.path.join(directory, entry)
        # Determine the branch connector
        if index == entries_count - 1:
            connector = "└── "
            extension = "    "
        else:
            connector = "├── "
            extension = "│   "
        
        print(prefix + connector + entry)
        
        # If the entry is a directory, recursively print its contents
        if os.path.isdir(path):
            print_tree(path, prefix + extension)

In [13]:
print_tree(PROCESSED_DATA_PATH)

├── .ipynb_checkpoints
├── config
│   ├── .ipynb_checkpoints
│   │   └── config-checkpoint.json
│   ├── config.json
│   └── training_config.json
├── gnn
│   ├── test
│   │   ├── edges
│   │   │   ├── .ipynb_checkpoints
│   │   │   │   └── node_to_node-checkpoint.csv
│   │   │   └── node_to_node.csv
│   │   └── nodes
│   │       ├── .ipynb_checkpoints
│   │       │   └── node_label-checkpoint.csv
│   │       ├── node.csv
│   │       └── node_label.csv
│   └── train_gnn
│       ├── edges
│       │   └── node_to_node.csv
│       └── nodes
│           ├── node.csv
│           ├── node_label.csv
│           └── offset_range_of_training_node.json
└── xgb
    ├── feature_info.json
    ├── test.csv
    └── training.csv


In [16]:
# ---------------------------
# Create training_config.json (NGC container expects this)
# ---------------------------
fraud_count = int(train_df['isFraud'].sum())
non_fraud_count = int(len(train_df) - fraud_count)
scale_pos_weight = float(non_fraud_count) / max(1.0, float(fraud_count))
# "output_dir": "/trained_models"   # container path -> maps to MODEL_OUTPUT_PATH on host /opt/ml/model
scale_pos_weight

27.434310083918007

In [15]:
! aws s3 cp --recursive {PROCESSED_DATA_PATH} {S3_PREPROCESS_DATA_PATH}
! aws s3 ls --recursive {S3_PREPROCESS_DATA_PATH}

upload: processed/ieee-fraud-detection/config/.ipynb_checkpoints/config-checkpoint.json to s3://sagemaker-us-east-1-176843580427/processed/ieee-fraud-detection/config/.ipynb_checkpoints/config-checkpoint.json
upload: processed/ieee-fraud-detection/config/training_config.json to s3://sagemaker-us-east-1-176843580427/processed/ieee-fraud-detection/config/training_config.json
upload: processed/ieee-fraud-detection/config/config.json to s3://sagemaker-us-east-1-176843580427/processed/ieee-fraud-detection/config/config.json
upload: processed/ieee-fraud-detection/gnn/test/nodes/.ipynb_checkpoints/node_label-checkpoint.csv to s3://sagemaker-us-east-1-176843580427/processed/ieee-fraud-detection/gnn/test/nodes/.ipynb_checkpoints/node_label-checkpoint.csv
upload: processed/ieee-fraud-detection/gnn/test/nodes/node_label.csv to s3://sagemaker-us-east-1-176843580427/processed/ieee-fraud-detection/gnn/test/nodes/node_label.csv
upload: processed/ieee-fraud-detection/gnn/test/edges/.ipynb_checkpoints/